In [29]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt
import json
import datetime
import pyproj
import os
os.chdir("D:/Projects/crime-prediction")
pyproj.datadir.set_data_dir("D:/ProgramData/anaconda3/envs/crime-prediction/Library/share/proj")

In [30]:
pre_processed_file_path = 'data/preprocessed/preprocessed-6.csv'

In [31]:
def parse_preprocessed_data():
    df = pd.read_csv(
                         pre_processed_file_path,
                         parse_dates=['Offense Date'],                       
                         dtype={
                             'Precinct': 'category',
                             'Offense Category': 'category',
                             'Latitude': np.float32,
                             'Longitude': np.float32
                         }
                        )
    df["Id"] = df.index
    gdf = gpd.GeoDataFrame(df, geometry= df["geometry"].apply(wkt.loads), crs='EPSG:4326')
    return gdf

In [32]:
crime_gdf = parse_preprocessed_data()
print(len(crime_gdf))

1280004


In [33]:
print(crime_gdf.columns)

Index(['Report Number', 'Offense Date', 'Latitude', 'Longitude', 'Beat',
       'Precinct', 'Sector', 'Neighborhood', 'Reporting Area',
       'Offense Category', 'geometry', 'centroid', 'Temperature', 'Wind Speed',
       'Precipitation', 'Visibility', 'Tract Name_left', 'Tract Population',
       'Tract Population Below Poverty', 'White_Total', 'Black_Total',
       'Color_Total', '18_Under_Total', '65_Over_Total', 'StreetLight Count',
       'Tract Name_right', 'HouseHold Income', 'Id'],
      dtype='object')


In [34]:
crime_gdf["HouseHold Income Level"] = pd.qcut(crime_gdf['HouseHold Income'], q=5, labels=['Q1 (lowest)', 'Q2', 'Q3', 'Q4', 'Q5 (highest)'])


In [47]:
reporting_rates = crime_gdf.groupby(["HouseHold Income Level", "Year"], observed=False) \
                           .apply(lambda  x: (len(x)/x.groupby('Tract Name_left')['Tract Population'].mean().sum()) * 1000) \
                           .rename('reports_per_1000')

C:\Users\Vimil\AppData\Local\Temp\ipykernel_7868\1669089214.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda  x: (len(x)/x.groupby('Tract Name_left')['Tract Population'].mean().sum()) * 1000) \


In [50]:
pd.set_option('display.max_rows', None)
print(reporting_rates)

HouseHold Income Level  Year
Q1 (lowest)             2008    189.289984
                        2009    173.619032
                        2010    179.812636
                        2011    170.544855
                        2012    158.815265
                        2013    172.002718
                        2014    186.179805
                        2015    178.194742
                        2016    181.148034
                        2017    188.398066
                        2018    194.300847
                        2019    177.992873
                        2020    184.424820
                        2021    194.270464
                        2022    191.926034
                        2023    173.955089
                        2024    182.477699
                        2025     92.272637
Q2                      2008    140.628388
                        2009    146.246874
                        2010    147.321811
                        2011    143.797790
                        2

In [38]:
crime_gdf["Year"] = crime_gdf['Offense Date'].dt.year

In [49]:
pd.set_option('display.max_rows', None)
print(crime_gdf)

KeyboardInterrupt: 

In [43]:
 print(crime_gdf.groupby("Year").size())

Year
2008    72404
2009    74394
2010    72493
2011    68313
2012    67104
2013    72358
2014    77410
2015    75911
2016    77602
2017    78451
2018    80135
2019    72692
2020    76437
2021    70405
2022    73762
2023    69687
2024    68723
2025    31723
dtype: int64


In [44]:
print(crime_gdf['Tract Population'].mean())

4517.209466766826
